In [1]:
import pandas as pd
import numpy as np
import random as rnd
import scipy.stats as stats
import scipy.optimize as opt
from types import FunctionType
import json as json
import matplotlib as mpl
from math import exp
from matplotlib import pyplot as plt

rnd.seed(2)
import warnings
warnings.filterwarnings('ignore')

# 0. Introduction

Following the instructions in the assignment, the code below (1) assigns values to underlying parameters, using the same assumptions as in the "John Rust 1987 Python" Notebook and shows where the values for choice probabilities appear from, (2) simulates transition path of a single bus taking choice probabilities as given and their logarithm equal to the relative expected value of replacement decision; (3) calculates the simulated relative expected value of replacing the bus engine in the first period that results from the path and compares it to the one taken as given.  

As is evident from the description, we do not simply assume all parameters as given, but rather choose to keep some parts of the original code, where the values are derived. This, in conjunction with extensive comments and function descriptions, is aimed at assuring that the reader always knows where these values come from.

# 1. Assigning values to underlying parameters

## 1.1. Initializing Parameters

We start with setting the parameters for milage transition of buses. Assuming that the milage follows a truncated normal distribution with the mean of 6000 and standard deviation of 4000, we can calculate transition probabilities from the cumulative distribution function of the milage.  

In [2]:
#arbitrarily choosen parameteres
lower, upper = 0, 15000
mu, sigma = 6000, 4000
mileage_dist = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)

In [3]:
#calculating transition probabilities
p_x0 = mileage_dist.cdf(5000)
p_x1 = mileage_dist.cdf(10000) - p_x0
p_x2 = 1 - p_x1 - p_x0
p = (p_x0, p_x1, p_x2)

Again, we use the assumptions on replacement cost ($rc$), linear cost parameter ($\theta_{11}$) and discounting parameter ($\beta$) from the original notebook replicating Rust (1987). 

In [4]:
rc = 20
theta1_1 = 0.5
beta = 0.75

## 1.2. Defining Functions Used for Data Generation 

To calculate the costs of each decision in each state a general myopic costs function is defined, and takes the form and parameters of the cost function, the number of states and transition probabilities as arguments. 

In [5]:
def myopic_costs(S, MF, params, p):
    """
    This function computes the myopic expected cost associated with each decision for each state, 
    and returns an array of state/decision costs.
    
    Takes:
        * An integer S, describing the possible states of the bus
        * A maintenance cost function MF, which takes a vector of parameters and a `state' argument
        * A vector params, to be supplied to the maintenance cost function MF. The first element of 
          the vector is the replacement cost rc.
        * A (3x1) vector p describing the state transitions probabilities 
        
    Returns:
        * A (Nx2) array containing the maintenance and replacement costs for the N possible states of the bus
    """
    rc = params[0]
    maint_cost = [MF(s, params[1:]) for s in range(0, S)]
    repl_cost = [rc for state in range(0, S)]
    return np.vstack((maint_cost, repl_cost)).T

Following the assignment, we assume linear costs of maintenance, and the following function is used to calculate them for any state s. 

In [6]:
def lin_cost(s, params):
    
    """
    This function computes the maintenance cost using the parameter theta_1_1 and the state. 
    If number of parameters supplied is wrong, it throws an error. 
    
    Takes:
        * An integer s, describing current state of the bus
        * A vector params, where the second element is the parameter of the linear cost function theta_1_1 
                 
    Returns:
        * An integer equal to maintenance cost of bus engine in the current state.
    """
    try:
        theta1_1, = params
        return s*theta1_1
    except ValueError:
        raise ValueError
        print("Wrong number of parameters specified: expected 2, got {}".format(len(params)))

The utility function of the decision maker for a single time period is:

$$ u(x_{t}, i , \theta_{1})  = \left\{
  \begin{array}{l l}
    \qquad \enspace -c(x_t, \theta_1) + \epsilon_t(0)& \quad \text{if } i = 0\\
    -RC -c(0, \theta_1) + \epsilon_t(1) & \quad \text{if } i = 1
  \end{array} \right. \quad \text{(Errors are I.I.D. standard Gumbel)}$$

Assuming logistic utility and normalizing the value, we can calculate probability of replacing the engine using the following function:

In [7]:
def choice_prob(cost_array):
    """
    Returns the probbability of each choice, conditional on an array of state/decision costs.
    """
    S = cost_array.shape[0]
    cost = cost_array - cost_array.min(1).reshape(S, -1)
    util = np.exp(-cost)
    pchoice = util/(np.sum(util, 1).reshape(S, -1))
    return pchoice

Finally, to derive conditional choice probabilities we require a contraction mapping function: 

In [8]:
def contraction_mapping(
    S, p, MF, params, beta=0.75, threshold=1e-6, suppr_output=False
):
    """
    Compute the non-myopic expected value of the agent for each possible decision and each possible
    state of the bus.
    Iterate until the difference in the previously obtained expected value and the new expected value
    is smaller than a constant.
    Takes:
        * A finite number of states S
        * A state-transition probability vector p = [p(0), p(1), p(2), ..., p(k)] of length k < N
        * A maintenance cost function MF
        * A vector params for the cost function
        * A discount factor beta (optional)
        * A convergence threshold (optional)

    Returns:
        * The converged choice probabilities for the forward-looking and myopic agents for each state,
        conditional on `params'
    """
    achieved = True
    # Initialization of the state-transition matrices: describe the state-transition probabilities
    # if the maintenance cost is incurred, and regenerate the state to 0 if the replacement cost
    # is incurred.
    ST_mat = np.zeros((S, S))
    p = np.array(p)
    for i in range(S):
        for j, _p in enumerate(p):
            if i + j < S - 1:
                ST_mat[i + j][i] = _p

            elif i + j == S - 1:
                ST_mat[S - 1][i] = p[j:].sum()
            else:
                pass

    R_mat = np.vstack((np.ones((1, S)), np.zeros((S - 1, S))))

    # Initialization of the expected value (which is also the myopic
    # decision cost of the agent). Here, the forward-looking component is initialized at 0.
    k = 0
    EV = np.zeros((S, 2))
    EV_myopic = EV_new = myopic_costs(S, MF, params, p)
    # Contraction mapping loop
    while abs(EV_new - EV).max() > threshold:
        # Store the former expected value
        EV = EV_new
        # Obtained the probability of maintenance and replacement from the former expected value
        pchoice = choice_prob(EV)
        # Compute the expected cost for each state: Nx1 vector
        ecost = (pchoice * EV).sum(1)
        # Compute the two components of forward-looking utility: In case of maintenance,
        # utility of future states weighted by transition probabilities. In case of replacement,
        # the future utility is the utility of state 0
        futil_maint = np.dot(ecost, ST_mat)
        futil_repl = np.dot(ecost, R_mat)
        futil = np.vstack((futil_maint, futil_repl)).T
        # Future utility is discounted by beta, and added to the myopic cost.
        EV_new = EV_myopic + beta * futil
        k += 1
        if k == 1000:
            achieved = False
            break

    if not suppr_output:
        if achieved:
            print("Convergence achieved in {} iterations".format(k))
        else:
            print(
                "CM could not converge! Mean difference = {:.6f}".format(
                    (EV_new - EV).mean()
                )
            )

    return (choice_prob(EV_new), choice_prob(EV_myopic))

## 1.3. Deriving Choice Probabilities

Thus, the choice probabilities for each possible state of the bus are calculated with the contraction mapping algorithm, and when performing the simulation we will use the resulting vector as a vectir of relative expected values of replacement. We set the number of states to 70, like in the original replication of Rust (1987).

In [9]:
#assign values to corresponding arguments of functions
params_lin = (rc, theta1_1)
p = (p_x0, p_x1, p_x2)

# create an array with probabilities using contraction mapping
lin_forward, _ = contraction_mapping(
    S=70, p=p, MF=lin_cost, params=params_lin, beta=0.75
)
pchoice = lin_forward.T[0]

Convergence achieved in 54 iterations


# 2. Simulating transition path of a single bus

## 2.1. Define functions

We first defined the transition function that generates the new variables for the t+1 period from the values in the t period. 

Using the following equation we derive the relative expected value of replacement from the choice probability.:

$$ ln Pr\{a_{t} | x_{t}\} = v(x_{t}, a_{t}) - v ( x_{t}, 0)  $$

Assuming a distribution of shocks corresponding to each of the two decision in each state t: replacing ($\epsilon_{1t}$) and maintaining ($\epsilon_{0t}$) - to be of the form $F(x)=\exp(-\exp(-x)))$, we simulate replacement decisions, based on the following rule:
 
$$ i_{t} = \left\{ \begin{array}{rcl} 1 & \mbox{if} & v(x_{t}, 1) - v ( x_{t}, 0) + \epsilon_{t}(1) - \epsilon_{t}(0) \geq 0 \\  
0  & \mbox{if} & otherwise
\end{array}\right. $$


In [10]:
def transition(bus_array, p):
    """
    Return the updated bus dataset after one decision of our agent.
    Takes:
        * bus_array : An array of buses, containing the identifier of the buses, their mileages, and their current
                        state and random variables from the standard type I exteme value distribution.
        * p: The converged choice probabities of the agent making the decision

    Returns:
        * The updated dataset of buses, with the new decisions appended at the end of the dataframe.
    """
    # Recovering the number of buses, the previous mileage and the previous states of the buses
    n_bus = int(bus_array[:, 0].max())
    prev_mileage = bus_array[-n_bus:, 2]
    prev_states = bus_array[-n_bus:, 3]
    prev_choice = bus_array[-n_bus:, 1]

    # Generating the new mileage and state
    new_mileage = (1 - prev_choice) * prev_mileage + mileage_dist.rvs(size=n_bus)
    new_states = np.floor(new_mileage / 5000)
    # Add random variables from the standard type I extreme value distribution
    new_shocks = np.random.gumbel(size=(2, n_bus))

    # Use choice probabilities to compute the relative expected value of replacement
    relative_expected_value_of_replacement = np.log(1 - pchoice[int(new_states[0])])

    # Simulate replacement decisions:
    if (
        relative_expected_value_of_replacement + (new_shocks[1] - new_shocks[0])[0]
    ) >= 0:
        current_choice = 1
    else:
        current_choice = 0

    # Save everything in a new array
    new_array = np.vstack(
        (
            bus_array[-n_bus:, 0],
            current_choice,
            new_mileage,
            new_states,
            new_shocks,
            relative_expected_value_of_replacement,
        )
    )

    return np.vstack((bus_array, new_array.T))

We then defined a function (calculate_utility) to calculate the net present value of realized payoffs. For this, we use the above definced myopic_cost function to calculate the replacement and maintenance costs for each period, given the previously definded paramteres, linear cost function and state. We follow by computing the utility for each period based on choice, costs and random shocks. Finally, using the beta paramters we calculate the net present value of realized payoffs for the bus. 

In [11]:
def calculate_utility(
    bus_array: np.ndarray, cost_function: FunctionType, parameters: tuple, beta: float
) -> (float, pd.DataFrame):

    """
    Calculates the net present value of realized payoffs.
    Takes:
        * bus_array: An array of buses, containing the identifier of the buses, their mileages,
                     and their current state and random variables from the standard type I
                     exteme value distribution.
        * cost_function: A maintenance cost function MF, which takes a vector of parameters and
                         a `state' argument.
        * parameters: A vector params, to be supplied to the maintenance cost function MF.
                      The first element of the vector is the replacement cost rc.
        * beta: Discount factor.
    Returns:
        * net present value of realized payoffs
        * dataframe of relative expected values, simulated choices, costs and utilities

    """

    # create Pandas Dataframe from bus array
    df = pd.DataFrame(
        bus_array,
        columns=[
            "Identifier",
            "Choice",
            "Mileage",
            "State",
            "ϵ0",
            "ϵ1",
            "relative EV(repl)",
        ],
    )

    # add discount factor
    df["β"] = beta

    # Create t - time periods
    df["t"] = df.index + 1

    # For each possible State calculate the cost of maintenance
    maintenance_cost = myopic_costs(
        int(df["State"].max() + 1), cost_function, parameters, _
    ).T[0]

    Maintenance_Cost = (
        pd.DataFrame(maintenance_cost)
        .reset_index()
        .rename(columns={"index": "State", 0: "maintenance_cost"})
    )

    # For each possible State calculate the cost of replacement
    replacement_cost = myopic_costs(
        int(df["State"].max() + 1), cost_function, parameters, _
    ).T[1]

    Replacement_Cost = (
        pd.DataFrame(replacement_cost)
        .reset_index()
        .rename(columns={"index": "State", 0: "replacement_cost"})
    )

    # Merge maintenance and replacement consts to realized States
    df = df.merge(Maintenance_Cost, on="State", how="left").merge(
        Replacement_Cost, on="State", how="left"
    )

    # Calculate utilities for each period based on choice, cost and random shock
    df = df.assign(
        util=lambda x: np.where(
            x["Choice"] == 0,
            -1 * (x["maintenance_cost"]) + x["ϵ0"],
            -1 * (x["replacement_cost"]) + x["ϵ1"],
        )
    )

    # drop the first column that corresponds to the first period of choice
    df = df.loc[lambda x: x["t"] > 1]

    # Return the net present value of realized payoff, and the whole dataframe
    return ((df["β"] ** (df["t"] - 1)) * df["util"]).sum(), df

## 2.2. Generate data 100 times and simulate decisions

Now that we defined all the necessary functions, we generate the data for one bus and a 1000 periods for two different scenarios: when the decision in the first period is to replace the bus and when the decision is to not replace the bus. 

In [12]:
n_bus = 1
#We predefine the inital shocks so it will be the same for each repetition and for both inital decisions
initial_shocks = np.random.gumbel(size=(n_bus, 2))

Intitializing the bus for the first period with $i_1=0$ (first decision is to not replace)

In [13]:
# The bus array is a n_bus (here 1) x 7 array, where
# the first element is the bus number (this will be 1 througout, since the number of busses is only 1)
# the second element is the choice in the given period: 1 if replaced 0 if not
# the third element is the mileage of the bus
# the fourth element is the State which is a discretized value of the miles variable
# the fifth element is the realtive excepted value of replacement for the given state derived from the choice probabilities
# the last two elements are two shocks, one for no replacement and one for replacement
# ["Identifier", "Choice", "Mileage", "State", "ϵ0", "ϵ1", "relative EV(repl)"]
init_bus_array_0 = np.hstack(
    (
        np.linspace(1, n_bus, n_bus).reshape(-1, 1),
        np.zeros((n_bus, 3)),
        initial_shocks,
        np.zeros((n_bus, 1)),
    )
)

We generate the entire data (1000 periods) with $i_1=0$ and calculate the net present value of realized payoff 100 times.

In [14]:
U0 = []
for i in range(1, 101):
    n_periods = 1000
    bus_array_0 = init_bus_array_0.copy()
    for j in range(n_periods):
        bus_array_0 = transition(bus_array_0, pchoice)
    u, lin_df_ba0 = calculate_utility(bus_array_0, lin_cost, params_lin, beta)
    U0.append(u)

Intitializing the bus for the first period with $i_1=1$ (first decision is to replace)

In [15]:
init_bus_array_1 = np.hstack(
    (
        np.linspace(1, n_bus, n_bus).reshape(-1, 1),
        np.ones((n_bus, 1)),
        np.zeros((n_bus, 2)),
        initial_shocks,
        np.zeros((n_bus, 1)),
    )
)

We generate the entire data (1000 periods) with $i_1=1$ and calculate the net present value of realized payoff 100 times.

In [16]:
U1 = []

for i in range(1, 101):
    n_periods = 1000
    bus_array_1 = init_bus_array_1
    for j in range(n_periods):
        bus_array_1 = transition(bus_array_1, pchoice)
    u, lin_df_ba1 = calculate_utility(bus_array_1, lin_cost, params_lin, beta)
    U1.append(u)

# 3. Calculating the simulated relative expected value of replacement

We compute the mean net present value for both intial decisions which approximates $\beta E[V_{\theta} (x_2, \epsilon_2) | x_1, i]$ for $t=1, i=0$ and for $t=1, i=1$

In [17]:
U0_mean=np.mean(U0)
U1_mean=np.mean(U1)
print("Average NPV when first decision is to not replace: ", U0_mean)
print("Average NPV when first decision is to replace: ", U1_mean)

Average NPV when first decision is to not replace:  -4.979817995052038
Average NPV when first decision is to replace:  -4.98660564324068


We calculate the utility of replacement and not replacement in the first period using the utility function of the decision maker for a single time period (defined in part 1.2.). The state is 0, since the bus starts with zero milage in the first period. 
$$ u(0, i , \theta_{11})  = \left\{
  \begin{array}{l l}
    \qquad \enspace -\theta_{11} \cdot 0 + \epsilon_t(0)& \quad \text{if } i = 0\\
    -RC -\theta_{11} \cdot 0 + \epsilon_t(1) & \quad \text{if } i = 1
  \end{array} \right. $$

In [18]:
u_1_0 = theta1_1 * 0 + init_bus_array_0.T[4]
print("Utility for the first period if no replacement: ", u_1_0[0])
u_1_1 = theta1_1 * 0 + init_bus_array_1.T[5] - rc
print("Utility for the first period if replacement: ", u_1_1[0])

Utility for the first period if no replacement:  2.4640289217422455
Utility for the first period if replacement:  -19.988091756276344


Finally, we compute the relative expected value of replacing the engine using the following formula:
$$ u(x_1, 1) + \tfrac{1}{100}\sum^{100}_{k=1} U^{(1)}_k - u(x_1, 0) - \tfrac{1}{100}\sum^{100}_{k=1} U^{(0)}_k $$

In [19]:
rel_EV_repl = u_1_1 + U1_mean - u_1_0 - U0_mean
print("Simulated relative expected value of replacing the engine in the first period: ", rel_EV_repl[0])
print("Simulated probability of replacing the engine in the first period: ", exp(rel_EV_repl))

Simulated relative expected value of replacing the engine in the first period:  -22.45890832620723
Simulated probability of replacing the engine in the first period:  1.7628690145258924e-10


### Comapring to the precise value

The precise value of $ v(x_{t}, 1) - v ( x_{t}, 0)  $ is $ ln\{ Pr\{1 | x_{t}\} $
For $ x_{t} = 0 $ this is the log of the propbability of replacement in the first state. 

In [20]:
rel_EV_rep_0 = np.log(1 - pchoice[0])
print("Precise value of realtive expected value of replacing the engine in the first period: ", rel_EV_rep_0)
print("Precise probability of replacing the engine in the first period: ", 1-pchoice[0])

Precise value of realtive expected value of replacing the engine in the first period:  -18.821945967752466
Precise probability of replacing the engine in the first period:  6.694724774547467e-09


The simulated relative expected value and probability of replacement in the first period is very close to the precise ones.  This large negative expected value and very small probability come from the fact that it is very unlikely that replacing a new engine and only saving one increment of maintenance cost is an optimal decision. The agent knows that he will behave optimal in the future and will replace the engine once it is optimal to replace it, thus he will not want to replace it in period one with $ x_t = 0 $. The averge NPV for replacement and no replacement in the first period is very similar, which also shows that the agent would not gain significant utility in the future by paying high replacement costs in the first period. 